# Session 8: Abstract Classes, Interfaces, and Error Handling

We've talked about the `protected` keyword and seen some interfaces in previous sessions when dealing with `IEnumerable` but haven't gotten into the science of Object Oriented Programming (OOP) and how we can create a hierarchy of objects that we can work with in our applications.  In some of the .NET frameworks like ASP.NET Core, interfaces and abstract classes are used in many places and are key to your successful use of the framework.

## Abstract Classes

Everything in C# is an object.  A class in C# can derive from another class in a **is a** relationship.  Let's look at the common Shape sample:

In [1]:
abstract class Shape {
    
}

class Triangle : Shape { }
class Rectangle : Shape { }
class Square : Rectangle { }

The below script needs to be able to find the current output cell; this is an easy method to get it.

In C#, we declare a class is abstract with the `abstract` keyword.  This prevents anyone from being able to create this type of class directly and they can only `inherit` from it using the `: ClassName` notation you see in `Triangle`, `Rectangle`, and `Square`.  `Square` inherits from Rectangle so a `Square` **is a** `Rectangle` and also a `Shape`

We can add methods to the `Shape` and they will be available in all of the other classes: 

In [2]:
abstract class Shape {
    public int Perimeter() { return 10; }
    public decimal Area() { return 15; }
}

class Triangle : Shape { }
class Rectangle : Shape { }
class Square : Rectangle { }

// consequently, we can execute the perimeter and area methods on Triangle, Rectangle, and Square
var t = new Triangle();
display(t.Area());

15

However, we know that `Perimeter` and `Area` are calculated differently for each shape.  We can add properties specific to each class, declare the `Perimiter` and `Area` methods as `abstract` and `override` them in each of the derived shape classes.  An `abstract method` or `abstract property`:
- **Only appears** in an abstract class
- **MUST** be implemented in a class that inherits from it. 
- Provides no body definition in the abstract class

In [3]:
abstract class Shape {
    public abstract int Perimeter();
    public abstract decimal Area();
}

class Triangle : Shape { 
    public int Side1Length;
    public int Side2Length;
    public int Side3Length;
    
    public override int Perimeter() {
        return Side1Length + Side2Length + Side3Length;
    }
    
    public override decimal Area() {
        var semiPerimeter = 0.5 * Perimeter();
        
        // Use Heron's Formula
        return (decimal)Math.Sqrt(semiPerimeter * (semiPerimeter - Side1Length) * (semiPerimeter - Side2Length) * (semiPerimeter - Side3Length));
    }
    
}
class Rectangle : Shape {

    public int Length;
    public int Width;
    
    public override int Perimeter() {
        return Length*2 + Width*2;
    }
    
    public override decimal Area() {
        return Length * Width;
    }
    

}
class Square : Rectangle { }

// consequently, we can execute the perimeter and area methods on Triangle, Rectangle, and Square
var t = new Triangle() { Side1Length=3, Side2Length=4, Side3Length=5};
display(t.Area());
display(t.Perimeter());

6

12

You can declare variables or parameters of the base type and pass in or assign the derived type.  This gives your classes more flexibility in their interactions.

In [4]:
abstract class Shape {
    public int Perimeter() { return 10; }
    public decimal Area() { return 15; }
}

class Triangle : Shape { }
class Rectangle : Shape { }
class Square : Rectangle { }

Shape s = new Rectangle();
display(s.Area());

bool IsQuadrilateral(Shape s) {
    return (s is Rectangle);
}

Shape t = new Triangle();
display("Rectangle is quadrilateral: " + IsQuadrilateral(s));
display("Triangle is quadrilateral: " + IsQuadrilateral(t));

15

Rectangle is quadrilateral: True

Triangle is quadrilateral: False

### Virtual Methods

There are times when you want to provide a default implementation of a method, but provide the ability for subclasses to be able to override or add value to that method.  This is done through the use of the `virtual` keyword in the base class and the child classes use the `override` keyword or `new` keyword to re-implement those methods.  [New](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/new-modifier) will hide the base implementation and prevent any compiler errors.

In [5]:
abstract class Shape {
    public abstract int Perimeter();
    public virtual decimal Area() {
        return Length * Width;
    }

    public int Length;
    public int Width;
    
}

class Triangle : Shape { 
    public int Side1Length;
    public int Side2Length;
    public int Side3Length;
    
    public override int Perimeter() {
        return Side1Length + Side2Length + Side3Length;
    }
    
    public decimal Area() {
        var semiPerimeter = 0.5 * Perimeter();
        
        // Use Heron's Formula
        return (decimal)Math.Sqrt(semiPerimeter * (semiPerimeter - Side1Length) * (semiPerimeter - Side2Length) * (semiPerimeter - Side3Length));
    }
    
}
class Rectangle : Shape {

    public override int Perimeter() {
        return Length*2 + Width*2;
    }
    
}
class Square : Rectangle { }

// consequently, we can execute the perimeter and area methods on Triangle, Rectangle, and Square
var t = new Triangle() { Side1Length=3, Side2Length=4, Side3Length=5};
display(t.Area());
display(t.Perimeter());

6

12

### [Sealed Classes](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/abstract-and-sealed-classes-and-class-members)

You can prevent another class from inheriting a class by using the `sealed` keyword.  Members of the class that were previously marked virtual are no longer available to be overridden.

In [6]:
abstract class Shape {
    public abstract int Perimeter();
    public abstract decimal Area();

    public int Length;
    public int Width;
    
}

// Try marking this class as sealed
class Rectangle : Shape {

    public override int Perimeter() {
        return Length*2 + Width*2;
    }
    
    // Also try marking this method as sealed
    public override decimal Area() {
        return Length * Width;
    }
    
}
class Square : Rectangle { 

    public override decimal Area() {
        return Length*Length;
    }

}

var s = new Square() { Length=4, Width=4};
display(s.Area());

16

## [Interfaces](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/interfaces/)

Interfaces define behaviors that a class or a struct **MUST** implement.  An interface can contain methods, properties, events and indexers.  They can also contain static methods with an implementation.  We previously saw collection objects that implement the `IEnumerable<T>` interface that allows LINQ methods to be executed against them.  These members in the interface are declared without scope, as they are required to be accessible to any object that wishes to interact with it.

Assign one or multiple interfaces to a class or struct using the same `:` notation, separating each interface name with a comma.  You must then implement the content of the interface in your object's definition.

By convention, many C# developers name their interfaces starting with the letter I. 

In [7]:
interface IAuditableData {
    
    // Unique identifier for this object
    long ID { get; }
    
    // Person who wrote this object
    string Author { get; set; }
    
    // Modification dates and log id
    DateTime CreateDate { get; }
    DateTime LastUpdateDate { get; }
    long EventLogId { get; }
}

class BlogPost : IAuditableData {
    
    public BlogPost(long logId) {
        this.EventLogId = logId;
    }
    
    public long ID { get; }
    public string Author { get; set; }
    public string Content { get; set; }
    
    public DateTime CreateDate { get; }
    public DateTime LastUpdateDate { get; }
    public long EventLogId { get; }
    
}

class BlogComment : IAuditableData {
    
    public long ID { get; }
    public string Author { get; set; }
    public string Content { get; set; }
    public string AuthorEmail { get; set; }
    
    public DateTime CreateDate { get; }
    public DateTime LastUpdateDate { get; }
    public long EventLogId { get; }
    
}

long DisplayEventLog(IAuditableData audit) {
    // do something with the eventlog object
    return audit.EventLogId;
}

var post = new BlogPost(10);
display(DisplayEventLog(post));

10

### Explicit Implementation

Implementing methods of an interface without any modifiers in the previous example makes them visible to any interactions with the class type.  It is also possible to HIDE the interface so that it is only visible when the class is explicitly cast to the interface type.

In [15]:
public interface IDoAThing {
    void DoTheThing();
}

class PublicThing : IDoAThing {
    public void DoTheThing() {
        display("I did the thing!");
    }
}

class SecretThing : IDoAThing {
    void IDoAThing.DoTheThing() {
        display("Shh...  I did a thing");
    }
}

// let's try to DoTheThing
var p = new PublicThing();
p.DoTheThing();

I did the thing!

### Default Implementations

Interfaces can implement other interfaces and can also provide default implementations of their methods.  A classic example of this use is to create new versions of interfaces without breaking compatibility with older interfaces.

In [27]:
public interface IDoAThing2 : IDoAThing {
    void DoSomethingElse();
    void IDoAThing.DoTheThing() {
        display("This is my default thing");
    }
}

class NewThing : IDoAThing2 {
    public void DoSomethingElse() {
        display("Something else?");
    }
}

// let's try to DoTheThing
var p = new NewThing();
p.DoSomethingElse();
// p.DoTheThing();

Unhandled exception: (17,3): error CS1061: 'NewThing' does not contain a definition for 'DoTheThing' and no accessible extension method 'DoTheThing' accepting a first argument of type 'NewThing' could be found (are you missing a using directive or an assembly reference?)

## Error Handling + Exceptions

### try...catch

### Create our own Exception